# Visualização dos dados com SQL
    Para fazer as views optei por utilizar a ferramenta  sqlite3, pois, mantenho o desenvolviemento na ferramenta que ja possuo dominio, bem como está configurada e pode gerar outras analises com as consultas, alem de o sqlite3 pode acessar os dataframes armazedos temporariamente na memoria.

# Desafios abordados
  - Para cada companhia aérea trazer a rota mais utilizada com as seguintes informações:
    - Razão social da companhia aérea
    - Nome Aeroporto de Origem
    - ICAO do aeroporto de origem
    - Estado/UF do aeroporto de origem
    - Nome do Aeroporto de Destino
    - ICAO do Aeroporto de destino
    - Estado/UF do aeroporto de destino
  - Para cada aeroporto trazer a companhia aérea com maior atuação no ano com as seguintes informações:
    - Nome do Aeroporto
    - ICAO do Aeroporto
    - Razão social da Companhia Aérea
    - Quantidade de Rotas à partir daquele aeroporto
    - Quantidade de Rotas com destino àquele aeroporto
    - Quantidade total de pousos e decolagens naquele aeroporto

In [17]:
# importe das bibliotecas
import sqlite3
import numpy as np
import pandas as pd
import os
import ast
import json
import re

# Load das Tabelas
## união das tabelas pelo 

In [25]:
def snake_case(valor):
    valor = re.findall('[A-Z]*[^A-Z]*', valor) # dá um split nas letras maiúsculas
    if valor[0] == '_':
        valor[0] = ''
    valor = [x.lower() for x in valor]
    return '_'.join(valor)[:-1]

In [28]:
# Carregando os dados .json
caminho_arquivo = 'C:/Users/Didrich/Documents/Python Scripts/DataEnginnerTest/VRA/VRA_Brutos/VRA_20218.json'  

if not os.path.isfile(caminho_arquivo):
    exit(1)  

with open(caminho_arquivo, 'r', encoding='utf-8-sig') as arquivo:
    conteudo_arquivo = arquivo.read()
    df = json.loads(conteudo_arquivo)
    df_vra = pd.DataFrame(df)
    
# Aplicar a função a todas as colunas do DataFrame
df_vra.columns = [snake_case(col) for col in df_vra.columns]


# Alteração do nome icaoempresa_area para icao para analises a view
df_vra.rename(columns={'icaoempresa_aérea': 'icao'}, inplace=True)

# Verificar o resultado
df_vra.head()

,icao,número_voo,código_autorização,código_tipo_linha,icaoaeródromo_origem,icaoaeródromo_destino,partida_prevista,partida_real,chegada_prevista,chegada_real,situação_voo,código_justificativa
0,IPM,5555,0,N,SBGL,SBGR,2021-08-15 19:15:00,None,2021-08-15 20:10:00,None,NÃO INFORMADO,N/A
1,IPM,5555,0,N,SBGL,SBGR,2021-08-16 19:25:00,None,2021-08-16 20:25:00,None,NÃO INFORMADO,N/A
2,AEA,0057,0,X,LEMD,SBGR,2021-08-18 18:45:00,2021-08-18 19:19:00,2021-08-19 05:25:00,2021-08-19 04:51:00,REALIZADO,N/A
3,AEA,0057,0,X,LEMD,SBGR,2021-08-08 18:45:00,2021-08-08 19:08:00,2021-08-09 05:25:00,2021-08-09 05:13:00,REALIZADO,N/A
4,AEA,0057,0,X,LEMD,SBGR,2021-08-29 18:45:00,2021-08-29 18:54:00,2021-08-30 05:25:00,2021-08-30 04:59:00,REALIZADO,N/A


In [33]:
# Carrregamento dos outros dados em csv
df_anac = pd.read_csv('C:/Users/Didrich/Documents/Python Scripts/DataEnginnerTest/AIR_CIA/ANAC_tratado/df_anac.csv', sep=';')
df_anac.head()

,razão_social,icao,iata,cnpj,atividades_aéreas,endereço_sede,telefone,e-mail,decisão_operacional,data_decisão_operacional,validade_operacional
0,AEROSUL TÁXI AÉREO LTDA (EX.: AUSTEN TÁXI AÉRE...,ASO,2S,27.315.694/0001-02,"SERVIÇOS AÉREOS PÚBLICOS, TÁXI-AÉREO, TRANSPOR...","RODOVIA PR 218, KM 7, JARDIM UNIVERSITÁRIO, AE...",(43) 3176-4030,operacoes@aerosul.com.br,DECISÃO Nº 282,10/02/2021,NaN
1,ATA - AEROTÁXI ABAETÉ LTDA.,ABJ,NaN,14.674.451/0001-19,"TÁXI-AÉREO, TRANSPORTE AÉREO NÃO REGULAR, TRAN...","PÇA. GAGO COUTINHO, S/N, AEROPORTO INTERNACIO...",(71) 34629600 / (71) 996643563,guilherme@abaete.com.br,DECISÃO Nº 36,05/03/2020,NaN
2,OMNI TÁXI AÉREO S.A.,OMI,O1,03.670.763/0001-38,"AEROFOTOGRAFIA, AEROINSPEÇÃO, SERVIÇOS AÉREOS ...","AV. AYRTON SENNA, 2541 RUA F1 LOTE 40 - AEROPO...",(61) 98335-3444,lira@plfconsultoria.com.br,DECISÃO Nº 100,10/07/2019,12/07/2024
3,PRIME TÁXI AIR LTDA,NaN,NaN,23.568.370/0001-25,"TÁXI-AÉREO, TRANSPORTE AÉREO NÃO REGULAR, SERV...","AL. RIO NEGRO, 585 - BLOCO A - CJ.95 - SALA E,...",(11) 41958307 / (11) 41958308,mmatta@primefractionclub.com.br,DECISÃO Nº 237,17/12/2020,18/12/2025
4,SIDERAL LINHAS AÉREAS LTDA.,SID,0S,10.919.908/0001-57,"TRANSPORTE AÉREO NÃO REGULAR, TRANSPORTE AÉREO...","RODOVIA CONTORNO LESTE BR-116, Nº 9.119, 3º AN...",(41) 3116-9000 / (41) 3116-9059,office@siderallinhasaereas.com.br,DECISÃO Nº 37,05/03/2020,06/03/2025


In [34]:
df_aero = pd.read_csv('C:/Users/Didrich/Documents/Python Scripts/DataEnginnerTest/Aerodomos/Aerodomos_tratado/df_aero.csv', sep=';')
df_aero.head()

,id,iata,icao,name,location,street_number,street,city,county,state,country_iso,country,postal_code,phone,latitude,longitude,uct,website
0,8847.0,YYZ,CYYZ,Toronto Pearson International Airport,"Toronto, Ontario, Canada",6301,Silver Dart Drive,Mississauga,NaN,Ontario,CA,Canada,L5P 1B2,+1 416-247-7678,43.677720,-79.624820,-240.0,http://www.torontopearson.com/
1,53.0,ACC,DGAA,Kotoka International Airport,"Accra, Ghana",NaN,Airport Road,Accra,Accra Metropolis,Greater Accra Region,GH,Ghana,NaN,+233 30 277 6171,5.606095,-0.168124,0.0,http://www.gacl.com.gh/
2,3531.0,KAN,DNKN,Mallam Aminu Kano International Airport,"Kano, Nigeria",NaN,Sani Abacha Way,Kano,NaN,Kano,NG,Nigeria,NaN,NaN,12.045650,8.522076,60.0,http://www.faan.gov.ng/mallam-aminu-kano-inter...
3,4308.0,LOS,DNMM,Murtala Muhammed International Airport,"Lagos, Nigeria",NaN,NaN,Lagos,NaN,Lagos,NG,Nigeria,NaN,+234 1 493 3385,6.581819,3.321135,60.0,http://www.faan.gov.ng/mmia/
4,947.0,BRU,EBBR,Brussels Airport (Zaventem Airport),"Brussels, Belgium",NaN,Leopoldlaan,Zaventem,Vlaams-Brabant,Vlaanderen,BE,Belgium,1930,+32 2 753 77 53,50.900997,4.485574,120.0,http://www.brusselsairport.be/


# Views com sqllite3

## Rotas mais utilizadas

 ### Razão social da companhia aérea

In [35]:
import sqlite3
import pandas as pd

# Carregue os DataFrames em tabelas temporárias no SQLite
con = sqlite3.connect(':memory:')
df_vra.to_sql('voos', con, index=False)
df_anac.to_sql('companhias', con, index=False)
df_aero.to_sql('aeroportos', con, index=False)

# Crie uma view para obter a rota mais utilizada para cada companhia aérea
consulta = """
CREATE VIEW IF NOT EXISTS view_rotas_mais_utilizadas AS
SELECT
    c.razão_social AS 'Razão Social',
    v.icaoaeródromo_origem || ' - ' || v.icaoaeródromo_destino AS 'Rota',
    COUNT(*) AS 'Número de Voos'
FROM
    companhias c
JOIN
    voos v
ON
    c.icao = v.icao
GROUP BY
    c.razão_social, v.icaoaeródromo_origem, v.icaoaeródromo_destino
HAVING
    COUNT(*) = (
        SELECT
            MAX(v1)
        FROM
            (
                SELECT
                    COUNT(*) AS v1
                FROM
                    companhias c1
                JOIN
                    voos v1
                ON
                    c1.icao = v1.icao
                GROUP BY
                    c1.razão_social, v1.icaoaeródromo_origem, v1.icaoaeródromo_destino
            )
    )
"""

# Execute a consulta para criar a view
con.execute(consulta)

# Agora você pode consultar a view para obter a rota mais utilizada para cada companhia aérea
consulta_view = "SELECT * FROM view_rotas_mais_utilizadas"
resultado = pd.read_sql_query(consulta_view, con)

# Imprima o resultado
print(resultado)

# Feche a conexão com o SQLite
con.close()


                  Razão Social         Rota  Número de Voos
0  SIDERAL LINHAS AÉREAS LTDA.  SBRF - SBGR              69


 ### Nome Aeroporto de Origem

In [84]:
import sqlite3
import pandas as pd

# Carregue os DataFrames em tabelas temporárias no SQLite
con = sqlite3.connect(':memory:')
df_vra.to_sql('voos', con, index=False)
df_anac.to_sql('companhias', con, index=False)
df_aero.to_sql('aeroportos', con, index=False)

# Crie uma consulta para obter o nome do Aeroporto SBRF
consulta = """
SELECT
    name AS 'Nome do Aeroporto'
FROM
    aeroportos
WHERE
    icao = 'SBRF'
"""

# Execute a consulta
resultado = pd.read_sql_query(consulta, con)

# Imprima o resultado
print(resultado)

# Feche a conexão com o SQLite
con.close()


                                   Nome do Aeroporto
0  Recife/Guararapes–Gilberto Freyre Internationa...


### ICAO do aeroporto de origem

In [87]:
import sqlite3
import pandas as pd

# Carregue os DataFrames em tabelas temporárias no SQLite
con = sqlite3.connect(':memory:')
df_vra.to_sql('voos', con, index=False)
df_anac.to_sql('companhias', con, index=False)
df_aero.to_sql('aeroportos', con, index=False)

# Crie uma consulta para obter o nome e o ICAO do Aeroporto SBRF
consulta = """
SELECT
    icao AS 'ICAO do Aeroporto',
    name AS 'Nome do Aeroporto'
FROM
    aeroportos
WHERE
    icao = 'SBRF'
"""

# Execute a consulta
resultado = pd.read_sql_query(consulta, con)

# Imprima o resultado
print(resultado)

# Feche a conexão com o SQLite
con.close()


  ICAO do Aeroporto                                  Nome do Aeroporto
0              SBRF  Recife/Guararapes–Gilberto Freyre Internationa...


### Estado/UF do aeroporto de origem

In [88]:
import sqlite3
import pandas as pd

# Carregue os DataFrames em tabelas temporárias no SQLite
con = sqlite3.connect(':memory:')
df_vra.to_sql('voos', con, index=False)
df_anac.to_sql('companhias', con, index=False)
df_aero.to_sql('aeroportos', con, index=False)

# Crie uma consulta para obter o nome, ICAO e Estado/UF do Aeroporto SBRF
consulta = """
SELECT
    icao AS 'ICAO do Aeroporto',
    name AS 'Nome do Aeroporto',
    state AS 'Estado/UF'
FROM
    aeroportos
WHERE
    icao = 'SBRF'
"""

# Execute a consulta
resultado = pd.read_sql_query(consulta, con)

# Imprima o resultado
print(resultado)

# Feche a conexão com o SQLite
con.close()


  ICAO do Aeroporto                                  Nome do Aeroporto  \
0              SBRF  Recife/Guararapes–Gilberto Freyre Internationa...   

    Estado/UF  
0  Pernambuco  


### Nome do Aeroporto de Destino

In [93]:
import sqlite3
import pandas as pd

# Carregue os DataFrames em tabelas temporárias no SQLite
con = sqlite3.connect(':memory:')
df_vra.to_sql('voos', con, index=False)
df_anac.to_sql('companhias', con, index=False)
df_aero.to_sql('aeroportos', con, index=False)

# Crie uma consulta para obter o nome do Aeroporto SBGR
consulta = """
SELECT
    name AS 'Nome do Aeroporto'
FROM
    aeroportos
WHERE
    icao = 'SBGR'
"""

# Execute a consulta
resultado = pd.read_sql_query(consulta, con)

# Imprima o resultado
print(resultado)

# Feche a conexão com o SQLite
con.close()


                           Nome do Aeroporto
0  São Paulo–Guarulhos International Airport


### ICAO do Aeroporto de destino

In [92]:
import sqlite3
import pandas as pd

# Carregue os DataFrames em tabelas temporárias no SQLite
con = sqlite3.connect(':memory:')
df_vra.to_sql('voos', con, index=False)
df_anac.to_sql('companhias', con, index=False)
df_aero.to_sql('aeroportos', con, index=False)

# Crie uma consulta para obter o ICAO do Aeroporto SBGR
consulta = """
SELECT
    icao AS 'ICAO Aeroporto'
FROM
    aeroportos
WHERE
    icao = 'SBGR'
"""

# Execute a consulta
resultado = pd.read_sql_query(consulta, con)

# Imprima o resultado
print(resultado)

# Feche a conexão com o SQLite
con.close()


  ICAO Aeroporto
0           SBGR


### Estado/UF do aeroporto de destino

In [94]:
import sqlite3
import pandas as pd

# Carregue os DataFrames em tabelas temporárias no SQLite
con = sqlite3.connect(':memory:')
df_vra.to_sql('voos', con, index=False)
df_anac.to_sql('companhias', con, index=False)
df_aero.to_sql('aeroportos', con, index=False)

# Crie uma consulta para obter o Estado/UF do aeroporto de destino SBGR
consulta = """
SELECT
    state AS 'Estado/UF do Aeroporto de Destino'
FROM
    aeroportos
WHERE
    icao = 'SBGR'
"""

# Execute a consulta
resultado = pd.read_sql_query(consulta, con)

# Imprima o resultado
print(resultado)

# Feche a conexão com o SQLite
con.close()


  Estado/UF do Aeroporto de Destino
0                         São Paulo


## Companhia aérea com maior atuação no ano com as seguintes informações:


### Nome do Aeroporto

In [69]:
import sqlite3
import pandas as pd

# Adicione uma coluna de ano ao DataFrame df_vra para representar o ano
df_vra['ano'] = pd.to_datetime(df_vra['partida_prevista']).dt.year

# Carregue os DataFrames em tabelas temporárias no SQLite
con = sqlite3.connect(':memory:')
df_vra.to_sql('voos', con, index=False)
df_anac.to_sql('companhias', con, index=False)
df_aero.to_sql('aeroportos', con, index=False)

# Crie uma view para obter o ano com o maior número de voos para cada empresa
consulta = """
CREATE VIEW IF NOT EXISTS view_ano_mais_voos_por_empresa AS
WITH voos_por_ano AS (
    SELECT
        c.razão_social AS 'Razão Social da Companhia Aérea',
        v.ano AS 'Ano',
        COUNT(*) AS 'Número de Voos'
    FROM
        voos v
    JOIN
        companhias c
    ON
        v.icao = c.icao
    GROUP BY
        c.razão_social, v.ano
),
max_voos_por_empresa AS (
    SELECT
        rv.'Razão Social da Companhia Aérea',
        MAX(rv.'Número de Voos') AS 'Maior Número de Voos'
    FROM
        voos_por_ano rv
    GROUP BY
        rv.'Razão Social da Companhia Aérea'
)
SELECT
    vp.'Razão Social da Companhia Aérea',
    vp.Ano AS 'Ano com Mais Voos',
    vp.'Número de Voos'
FROM
    voos_por_ano vp
JOIN
    max_voos_por_empresa mv
ON
    vp.'Razão Social da Companhia Aérea' = mv.'Razão Social da Companhia Aérea'
    AND vp.'Número de Voos' = mv.'Maior Número de Voos'
"""

# Execute a consulta para criar a view
con.execute(consulta)

# Agora você pode consultar a view para obter o ano com o maior número de voos para cada empresa
consulta_view = "SELECT * FROM view_ano_mais_voos_por_empresa"
resultado = pd.read_sql_query(consulta_view, con)

# Imprima o resultado
print(resultado)

# Feche a conexão com o SQLite
con.close()


  Razão Social da Companhia Aérea  Ano com Mais Voos  Número de Voos
0            OMNI TÁXI AÉREO S.A.             2021.0              33
1     SIDERAL LINHAS AÉREAS LTDA.             2021.0             712


## ICAO do Aeroporto

In [79]:
import sqlite3
import pandas as pd

# Carregue os DataFrames em tabelas temporárias no SQLite
con = sqlite3.connect(':memory:')
df_vra.to_sql('voos', con, index=False)
df_anac.to_sql('companhias', con, index=False)
df_aero.to_sql('aeroportos', con, index=False)

# Crie uma consulta para obter o ICAO do aeroporto mais frequentado pela SIDERAL LINHAS AÉREAS LTDA.
consulta = """
SELECT
    a.icao AS 'ICAO Aeroporto',
    COUNT(*) AS 'Número de Voos'
FROM
    voos v
JOIN
    companhias c
ON
    v.icao = c.icao
JOIN
    aeroportos a
ON
    v.icaoaeródromo_origem = a.icao
WHERE
    c.razão_social = 'SIDERAL LINHAS AÉREAS LTDA.'
GROUP BY
    a.icao
ORDER BY
    COUNT(*) DESC
LIMIT 1
"""

# Execute a consulta
resultado = pd.read_sql_query(consulta, con)

# Imprima o ICAO do aeroporto mais frequentado
icao_aeroporto_mais_frequentado = resultado.iloc[0]['ICAO Aeroporto']
print("ICAO do Aeroporto mais frequentado pela SIDERAL LINHAS AÉREAS LTDA.:", icao_aeroporto_mais_frequentado)

# Feche a conexão com o SQLite
con.close()


ICAO do Aeroporto mais frequentado pela SIDERAL LINHAS AÉREAS LTDA.: SBGR


##  Razão social da Companhia Aérea

In [75]:
import sqlite3
import pandas as pd

# Carregue os DataFrames em tabelas temporárias no SQLite
con = sqlite3.connect(':memory:')
df_vra.to_sql('voos', con, index=False)
df_anac.to_sql('companhias', con, index=False)

# Crie uma consulta para obter o número de voos da "SIDERAL LINHAS AÉREAS LTDA."
consulta = """
SELECT
    c.razão_social AS 'Razão Social da Companhia Aérea',
    COUNT(*) AS 'Número de Voos'
FROM
    voos v
JOIN
    companhias c
ON
    v.icao = c.icao
WHERE
    c.razão_social = 'SIDERAL LINHAS AÉREAS LTDA.'
"""

# Execute a consulta
resultado = pd.read_sql_query(consulta, con)

# Imprima o resultado
print(resultado)

# Feche a conexão com o SQLite
con.close()


  Razão Social da Companhia Aérea  Número de Voos
0     SIDERAL LINHAS AÉREAS LTDA.             712


## Quantidade de Rotas à partir daquele aeroporto

In [78]:
import sqlite3
import pandas as pd

# Carregue os DataFrames em tabelas temporárias no SQLite
con = sqlite3.connect(':memory:')
df_vra.to_sql('voos', con, index=False)
df_anac.to_sql('companhias', con, index=False)
df_aero.to_sql('aeroportos', con, index=False)

# Crie uma consulta para obter o aeroporto mais frequentado pela "SIDERAL LINHAS AÉREAS LTDA."
consulta_aeroporto_mais_frequentado = """
SELECT
    a.icao AS 'ICAO Aeroporto'
FROM
    voos v
JOIN
    companhias c
ON
    v.icao = c.icao
JOIN
    aeroportos a
ON
    v.icaoaeródromo_origem = a.icao
WHERE
    c.razão_social = 'SIDERAL LINHAS AÉREAS LTDA.'
GROUP BY
    a.icao
ORDER BY
    COUNT(*) DESC
LIMIT 1
"""

# Execute a consulta para obter o ICAO do aeroporto mais frequentado
resultado_aeroporto = pd.read_sql_query(consulta_aeroporto_mais_frequentado, con)

# Imprima o ICAO do aeroporto mais frequentado
print("ICAO do Aeroporto mais frequentado:", resultado_aeroporto.iloc[0]['ICAO Aeroporto'])

# Agora, crie uma consulta para contar a quantidade de rotas saindo desse aeroporto
icao_aeroporto_mais_frequentado = resultado_aeroporto.iloc[0]['ICAO Aeroporto']
consulta_quantidade_rotas = f"""
SELECT
    COUNT(*) AS 'Quantidade de Rotas'
FROM
    voos
WHERE
    icaoaeródromo_origem = '{icao_aeroporto_mais_frequentado}'
"""

# Execute a consulta para obter a quantidade de rotas
resultado_quantidade_rotas = pd.read_sql_query(consulta_quantidade_rotas, con)

# Imprima a quantidade de rotas
print("Quantidade de Rotas a partir do Aeroporto:", resultado_quantidade_rotas.iloc[0]['Quantidade de Rotas'])

# Feche a conexão com o SQLite
con.close()


ICAO do Aeroporto mais frequentado: SBGR
Quantidade de Rotas a partir do Aeroporto: 8001


## Quantidade de Rotas com destino àquele aeroporto

In [80]:
import sqlite3
import pandas as pd

# Carregue os DataFrames em tabelas temporárias no SQLite
con = sqlite3.connect(':memory:')
df_vra.to_sql('voos', con, index=False)
df_aero.to_sql('aeroportos', con, index=False)

# Crie uma consulta para contar a quantidade de rotas com destino ao aeroporto SBGR
consulta = """
SELECT
    COUNT(*) AS 'Quantidade de Rotas'
FROM
    voos v
JOIN
    aeroportos a
ON
    v.icaoaeródromo_destino = a.icao
WHERE
    a.icao = 'SBGR'
"""

# Execute a consulta
resultado = pd.read_sql_query(consulta, con)

# Imprima a quantidade de rotas com destino ao aeroporto SBGR
quantidade_rotas_sbgr = resultado.iloc[0]['Quantidade de Rotas']
print("Quantidade de Rotas com destino ao Aeroporto SBGR (Guarulhos, São Paulo):", quantidade_rotas_sbgr)

# Feche a conexão com o SQLite
con.close()


Quantidade de Rotas com destino ao Aeroporto SBGR (Guarulhos, São Paulo): 8024


## Quantidade total de pousos e decolagens naquele aeroporto

In [81]:
import sqlite3
import pandas as pd

# Carregue os DataFrames em tabelas temporárias no SQLite
con = sqlite3.connect(':memory:')
df_vra.to_sql('voos', con, index=False)
df_aero.to_sql('aeroportos', con, index=False)

# Crie uma consulta para contar a quantidade total de pousos e decolagens no SBGR
consulta = """
SELECT
    (SELECT COUNT(*) FROM voos WHERE icaoaeródromo_origem = 'SBGR') AS 'Decolagens (Partidas)',
    (SELECT COUNT(*) FROM voos WHERE icaoaeródromo_destino = 'SBGR') AS 'Pousos (Chegadas)'
"""

# Execute a consulta
resultado = pd.read_sql_query(consulta, con)

# Imprima a quantidade total de pousos e decolagens no SBGR
total_decolagens = resultado.iloc[0]['Decolagens (Partidas)']
total_pousos = resultado.iloc[0]['Pousos (Chegadas)']

print("Quantidade total de Decolagens (Partidas) no SBGR:", total_decolagens)
print("Quantidade total de Pousos (Chegadas) no SBGR:", total_pousos)

# Feche a conexão com o SQLite
con.close()


Quantidade total de Decolagens (Partidas) no SBGR: 8001
Quantidade total de Pousos (Chegadas) no SBGR: 8024
